In [23]:
import torch
import coremltools as ct
from torch.optim.lr_scheduler import StepLR
import torch.nn as nn

class SitUpModel(nn.Module):
    def __init__(self, num_features, num_classes):
        super(SitUpModel, self).__init__()
        
        # Define the model layers
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=7, padding=2)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(128, num_classes)
        self.sm = nn.Softmax()
    
    def forward(self, x):
        # x shape: (batch_size, num_features, num_readings)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.pool(x)  # x shape: (batch_size, 128, 1)
        x = x.view(x.size(0), -1)  # Flatten to (batch_size, 128)
        x = self.fc(x)  # Output shape: (batch_size, num_classes)
        x = self.sm(x)
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: ", device)

# Load the checkpoint
checkpoint = torch.load('checkpoint.pth')

# Reinitialize the model and optimizer
model = SitUpModel(num_features=9, num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters())
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)

# Load the state dictionaries
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

model.eval()
sample_input = torch.rand(1,9,200).to(device)
traced_model = torch.jit.trace(model, sample_input)

# Convert to Core ML
mlmodel = ct.convert(
    traced_model,
    inputs=[ct.TensorType(shape=sample_input.shape)],
    convert_to="neuralnetwork"
)

# Save the Core ML model
mlmodel.save("Situps4.mlmodel")

/tmp/ipykernel_28498/3622926334.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('checkpoint.pth')
/home/thomas/Documents/ActiveAlarm_Processing/

device:  cuda


Running MIL frontend_pytorch pipeline:   0%|                                                                                                                                                                                             | 0/5 [00:00<?, ? passes/s]

Running MIL default pipeline:   0%|                                                                                                                                                                                                     | 0/86 [00:00<?, ? passes/s]/home/thomas/Documents/ActiveAlarm_Processing/conda/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:245: UserWarning: Input, 'x.1', of the source model, has been renamed to 'x_1' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
/home/thomas/Documents/ActiveAlarm_Processing/conda/lib/python3.10/site-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:273: UserWarning: Output, '66', of the source model, has been renamed to 'var_66' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Translating MIL ==> NeuralNetwork Ops: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████

In [19]:
import os

# Option 2: Relative to script location
script_dir = os.getcwd()
save_path = os.path.join(script_dir, "MyModel.mlpackage")

print(f"Attempting to save model to: {save_path}")

try:
    mlmodel.save(save_path)
    print(f"Model saved successfully to: {save_path}")
except Exception as e:
    print(f"Failed to save model: {e}")

Attempting to save model to: /home/thomas/Documents/ActiveAlarm_Processing/MyModel.mlpackage
Model saved successfully to: /home/thomas/Documents/ActiveAlarm_Processing/MyModel.mlpackage
